In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd

In [11]:
file_path = "data/requetes311.csv"
data = pd.read_csv(file_path)

#preview the csv file
data

/var/folders/y2/bxbn9j2n1nn68rb5sxx6lvhr0000gn/T/ipykernel_37713/2644414103.py:2: DtypeWarning: Columns (0: ID_UNIQUE, 1: TYPE_LIEU_INTERV, 2: RUE, 3: RUE_INTERSECTION1, 4: RUE_INTERSECTION2, 5: ARRONDISSEMENT, 6: ARRONDISSEMENT_GEO, 7: LIN_CODE_POSTAL, 8: DERNIER_STATUT) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


ID_UNIQUE                  343029
NATURE                     639090
ACTI_NOM                   639090
TYPE_LIEU_INTERV           343029
RUE                        296022
RUE_INTERSECTION1           51866
RUE_INTERSECTION2           51830
LOC_ERREUR_GDT             342800
ARRONDISSEMENT             343029
ARRONDISSEMENT_GEO         342786
LIN_CODE_POSTAL            268534
DDS_DATE_CREATION          639090
PROVENANCE_ORIGINALE       638675
PROVENANCE_TELEPHONE       639090
PROVENANCE_COURRIEL        639090
PROVENANCE_PERSONNE        639090
PROVENANCE_COURRIER        639090
PROVENANCE_TELECOPIEUR     639090
PROVENANCE_INSTANCE        639090
PROVENANCE_MOBILE          639090
PROVENANCE_MEDIASOCIAUX    639090
PROVENANCE_SITEINTERNET    639090
UNITE_RESP_PARENT          639090
LOC_LONG                   342800
LOC_LAT                    342800
LOC_X                      342800
LOC_Y                      342800
DERNIER_STATUT             343029
DATE_DERNIER_STATUT        639090
dtype: int64


In [3]:
#loader = CSVLoader(file_path=file_path)
#docs = loader.load_and_split()

In [4]:
import pandas as pd
from langchain_core.documents import Document

def create_metadata_chunks(df):
    metadata_docs = []
    
    for col in df.columns:
        # On récupère les valeurs uniques (limitées pour ne pas exploser le prompt)
        unique_values = df[col].unique()[:50].tolist() 
        count_unique = df[col].nunique()
        
        content = (
            f"Colonne: {col}\n"
            f"Nombre de valeurs uniques: {count_unique}\n"
            f"Exemples de valeurs: {', '.join(map(str, unique_values))}"
        )
        
        metadata_docs.append(Document(
            page_content=content, 
            metadata={"type": "schema", "column": col}
        ))
    return metadata_docs

docs = create_metadata_chunks(data)


In [5]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings
import faiss

# 1. Initialiser les embeddings Ollama
# Assurez-vous d'avoir fait : ollama pull nomic-embed-text
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# 2. Déterminer la dimension de l'index automatiquement

sample_embedding = embeddings.embed_query("test")
dimension = len(sample_embedding)

# 3. Créer l'index FAISS
index = faiss.IndexFlatL2(dimension)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [6]:
vector_store.add_documents(documents=docs)

['7e2eb41d-5dc1-45dd-a9be-06c4a7539cc4',
 '704040c9-d1ac-415f-8475-e9cf50a91688',
 '1e1bfff0-226c-43fa-be47-03d58c6abe3c',
 '8c130957-692c-4db3-9d78-26e165581d3e',
 'f721df29-1cb6-4b8d-8f11-37d8d5baa23a',
 'a2c1157f-2aff-4f68-bcda-96fbc8fc95e8',
 '0721f79b-9c42-4130-bee0-2f7d8806286d',
 '30616111-2dcc-4524-8edb-4546f3ddb9f2',
 '93fc7f4b-6fd5-46b1-9e82-36adca360acd',
 '8854e323-6a91-4f79-b0f4-487a7026a0d4',
 '6ee22a92-e7d2-440b-9d49-c338c35fec94',
 '35242b48-6059-45d6-b380-d2bd8e297f1e',
 'ab4410f1-1683-4ba6-9ad9-67c6fe2e2b7a',
 'e0d5d664-2edf-4cc5-a5b4-9a8cb0adf9d5',
 '1c53eb2b-ca36-4aaf-b73c-4b704cfb9f6d',
 '813740d6-b3cc-4ee0-9c69-366b003bc39c',
 '894d967a-ffda-4628-bc3c-ac41ac6e945d',
 'c3175c28-0cc8-41e9-b0a9-699123c7bf8f',
 'dbfaab78-a415-4d81-84bc-fcc10b0c9099',
 '011ce585-f47e-487e-a885-450a27a6512b',
 'db690efc-38df-4c8b-a5ea-0dad4409e5c8',
 '4d75e31d-4a7f-4143-a250-eff8cd65515a',
 '1a586e9a-ba34-4979-b75b-e4ba7ac7ff13',
 'fb56dc25-d606-4d94-a221-fbeee296469c',
 '8526db8e-90c8-

In [7]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
# 1. Initialiser le modèle LLM local via Ollama
# Vous pouvez changer "llama3" par "mistral" ou tout autre modèle téléchargé
llm = ChatOllama(model="gemma3:4b")

# 2. Configurer le retriever à partir de votre vector_store FAISS précédent
retriever = vector_store.as_retriever()

# 3. Définir le prompt système
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# 4. Créer la chaîne de documents (RAG)
# On passe ici le LLM Ollama défini plus haut
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# --- Exemple d'utilisation ---
# response = rag_chain.invoke({"input": "Quelle est la question ?"})
# print(response["answer"])

In [10]:
answer= rag_chain.invoke({"input": "Quels sont les appels les plus courants ?"})
answer['answer']

'Je ne peux pas répondre à cette question avec les informations fournies. Les données fournies listent uniquement les noms des colonnes et le nombre de valeurs uniques dans chaque colonne, ainsi que quelques exemples de valeurs.'

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--simple-csv-rag)